In [3]:
from rfpipe import candidates
import pylab as plt
import numpy as np
import glob
from loc_utils import get_maxsnr_cd, calc_cand_integration
%matplotlib inline

In [4]:
cd /Users/claw/data/realfast_candidates/21A-017

/Users/claw/data/realfast_candidates/21A-017


In [6]:
pkls = glob.glob('*pkl')
pkls

['cands_21A-017.sb39654566.eb40110122.59462.47288809028.26.1.pkl']

### Chose the pkl file to use 

In [7]:
pkl = pkls[0]

In [16]:
ccs = [cc for cc in candidates.iter_cands(pkl) if len(cc)]
print(ccs)

[CandCollection for 21A-017.sb39654566.eb40110122.59462.47288809028, scan 26, segment 5 with 1 candidate, CandCollection for 21A-017.sb39654566.eb40110122.59462.47288809028, scan 26, segment 37 with 1 candidate]


In [17]:
cc = ccs[0]


In [21]:
from rfpipe import reproduce

In [70]:
loc = [0, 21, 5, 1, 0]

In [71]:
cc.prefs.gainfile = './21A-017.sb39654566.eb40110122.59462.47288809028.GN'
cc.metadata.datasource = 'sdm'
cc.metadata.filename = 'realfast_21A-017.sb39654566.eb40110122.59462.47288809028_1630845033599'
cc.metadata.bdfstr = 'ASDMBinary/uid____evla_realfastbdf_1630845033599'

In [72]:
#cd = reproduce.pipeline_canddata(cc.state, loc)

In [73]:
import os

In [88]:
cc.prefs.fftmode='fftw'

In [89]:
sdmname = './realfast_21A-017.sb39654566.eb40110122.59462.47288809028_1630845033599'
print('{0}'.format('_'.join(os.path.basename(sdmname).split('_')[1:-1])))

21A-017.sb39654566.eb40110122.59462.47288809028


In [98]:
dm = cc.state.dmarr[cc.locs[0][2]]
cc2 = reproduce.refine_sdm('./realfast_21A-017.sb39654566.eb40110122.59462.47288809028_1630845033599', dm, gainpath='./', classify=False, refine=False, ddm=100, preffile='./realfast.yml', inprefs={'fftmode': 'fftw'})

2021-10-31 13:25:08,680 - root - INFO - Searching for the gainfile 21A-017.sb39654566.eb40110122.59462.47288809028.GN in ./
2021-10-31 13:25:08,708 - rfpipe.metadata - INFO - Reading metadata from ./realfast_21A-017.sb39654566.eb40110122.59462.47288809028_1630845033599, scan 1
2021-10-31 13:25:08,765 - rfpipe.preferences - INFO - Parsing preffile for preference set NRAOdefaultS
2021-10-31 13:25:08,792 - rfpipe.metadata - INFO - Reading metadata from ./realfast_21A-017.sb39654566.eb40110122.59462.47288809028_1630845033599, scan 1
2021-10-31 13:25:09,056 - rfpipe.state - INFO - Metadata summary:
2021-10-31 13:25:09,057 - rfpipe.state - INFO - 	 Working directory and fileroot: None, realfast_21A-017.sb39654566.eb40110122.59462.47288809028_1630845033599.1.1
2021-10-31 13:25:09,057 - rfpipe.state - INFO - 	 Using scan 1, source P331
2021-10-31 13:25:09,057 - rfpipe.state - INFO - 	 nants, nbl: 26, 325
2021-10-31 13:25:09,058 - rfpipe.state - INFO - 	 nchan, nspw: 560, 10
2021-10-31 13:25:09

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [8]:
cd = get_maxsnr_cd(pkl)

AssertionError: 

In [53]:
# cd = list(candidates.iter_cands(pkl, select='canddata'))[-1][1]
cd.snr1

4.9811177

In [ ]:
i, name, dm = calc_cand_integration(cd)

In [ ]:
print(i, name, dm)

In [ ]:
n = 10
i = i - 5
i, n

### CASA prep of slow and fast cutout SDMs

In [ ]:
# this will write integrations from i to i + n-1
ext = f'ddcut{i}to{i+n-1}_{dm}'
ddcut_name = name[:-1] + '.' + f'{ext}'
print(ddcut_name)

In [ ]:
!bdf_dedisperse_cut.py -i 1:{i}:{n} -d {dm} -e {ext} {name}

In [ ]:
!asdm2MS -s 1 --ocm co {ddcut_name}

### Check spectra of ddcut SDM
#### Add the correct path of gainfile

In [ ]:
from loc_utils import prepare_data

In [ ]:
sdmfile = glob.glob(f'{ddcut_name}')[0]
gainfile = '../FILE.GN'
delta_l = cd.l1
delta_m = cd.m1
spws = cd.state.spw

In [ ]:
data, st = prepare_data(sdmfile,
    gainfile,
    delta_l,
    delta_m,
    segment=0,
    dm=0,
    dt=1,
    spws=spws)

In [ ]:
d = np.real(data.mean(axis=1).mean(axis=2))

In [ ]:
len(st.freq), d.shape

#### set appropriate frequency range here

In [ ]:
mask = (st.freq < 3.5) & (st.freq > 2.5)

In [ ]:
d2 = d[:, mask]
ds = d2.mean(1)
amax = np.argmax(ds)

In [ ]:
plt.plot(ds)

In [ ]:
amax

In [ ]:
# plt.scatter(st.freq[mask], d2[amax-1, :], c='r')
# plt.scatter(st.freq[mask], d2[amax+1, :], c='g')
plt.scatter(st.freq[mask], d2[amax, :], c='k')

In [ ]:
for j in range(d2.shape[0]):
    plt.scatter(st.freq[mask], d2[j,:], label=f'{j}, {d[j,:][mask].mean()}')
plt.legend()

#### Delete the cutout SDMs

In [ ]:
ls -lrthd *{ddcut_name}*

In [ ]:
for f in glob.glob(f'{ddcut_name}*'):
    !rm -rf {f}

In [ ]:
ls -lrthd *{ddcut_name}*

#### Generate SDM with just the signal

#### set the appropriate (i, n) to use based on the above 2 plots

In [ ]:
i_use = i + 5

In [ ]:
i = i_use
n = 2

In [ ]:
ext = f'ddcut{i}to{i+n-1}_{dm}'
ddcut_name = name[:-1] + '.' + f'{ext}'
print(ddcut_name)

In [ ]:
!bdf_dedisperse_cut.py -i 1:{i}:{n} -d {dm} -e {ext} {name}

In [ ]:
!asdm2MS -s 1 --ocm co {ddcut_name}

In [ ]:
ls -dlrth {ddcut_name}*